In [ ]:
%autosave 5

Autosaving every 5 seconds


In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
import os
import re
import string
import pandas as pd
from functools import reduce
from math import log
import numpy as np

In [ ]:
!pip3 install transformers torch

     |████████████████████████████████| 1.3MB 8.6MB/s 
     |████████████████████████████████| 1.1MB 24.4MB/s 
     |████████████████████████████████| 890kB 32.3MB/s 
     |████████████████████████████████| 2.9MB 45.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=bbbd141b386a217bdaf26e9ccc1e5c78b344d64404e2fa9d5f7f47d49e90e77c
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
multi = pd.read_csv("/content/drive/MyDrive/Anushree/for_summar_bert.csv")
multi.head()

,Unnamed: 0,mixed_doc,health,sports,politics,food
0,361,"BACKGROUND.\nThe Citroen driver, competing in ...",the three-dimensional structures of proteins ...,"The Citroen driver, competing in the first of ...",NaN,BACKGROUND.\nRESULTS.\nCONCLUSIONS.
1,73,AwesomeAlways a crowd pleaser!<br />I love thi...,dna sequencing technologies have enabled us to...,It's expected to get signed by President Barac...,Congress's Christmas present to corporate Amer...,AwesomeAlways a crowd pleaser!<br />I love thi...
2,374,BACKGROUND.\nObama sounded skeptical about Key...,a key aim of evolutionary biology is to unders...,The 21-year-old was held by Northumbria Police...,Obama sounded skeptical about Keystone XL on t...,"BACKGROUND.\nEssentially, this is Canadian oil..."
3,155,"The GOP's latest Obamacare ""fix"" will be a tot...",that while working slightly more than 40 hours...,''The winner was very good today; we were seco...,"The GOP's latest Obamacare ""fix"" will be a tot...",I wish them all to have this feeling sometime .
4,104,BACKGROUND.\nWhy Congress is broken: tax exten...,"in the early eighteenth century, linnaeus prov...",Allardyce's current deal expires at the end of...,Why Congress is broken: tax extenders edition....,BACKGROUND.\nPOP Chips on the goIf you haven't...


In [ ]:
Data = multi

In [ ]:
import pandas as pd
#{'Health':0, 'Sports':1, 'Politics':2, 'Food':3}
Data = pd.DataFrame(k, 
               columns =['mixed_doc', 'health', 'sports', 'politics', 'food']) 
Data = Data.sample(frac = 1)
Data.head(5)


,mixed_doc,health,sports,politics,food
3,Obamacare cuts Medicaid doctor pay 42 percent ...,macroautophagy is an evolutionarily conserved...,Jon Paul Morosi is a national MLB writer for F...,Obamacare cuts Medicaid doctor pay 42 percent ...,BACKGROUND.\nMild sausagesI love this product ...
0,Very yummy!So far I've tried all of the flavor...,NaN,Just when it appeared the Los Angeles Lakers h...,"(I remember it in general, though.\nWhile ther...",Very yummy!So far I've tried all of the flavor...
2,Peanut Butter is back on the menu!Peanut butte...,NaN,Thirty years from now I dont think it will be ...,Can water markets help solve the American West...,Peanut Butter is back on the menu!Peanut butte...
4,"One is not born, but rather becomes, a citizen...",noncoding rnas are sequences that are transcr...,"On a side note, we feed Diamond Naturals Beef ...","One is not born, but rather becomes, a citizen...",BACKGROUND.\nCan't find the perfect Creme' Brl...
1,BACKGROUND.\ntwo-component regulatory systems ...,two-component regulatory systems are the most...,Ohio took away a womans welfare and food stamp...,"Last May, Kimberly Thompson, a warehouse worke...",BACKGROUND.\nGood Product- VERY Slow Super Sav...


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# initialize the model architecture and weights
model = T5ForConditionalGeneration.from_pretrained("t5-base")
# initialize the model tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-base")

In [ ]:
j=1
ind_accuracy=[]
true_label = []
pred_label = []
col1 = []
col2_0 = []
col2_1 = []
col2_2 = []
col2_3  = []
article_num=0
dic = {0: col2_0, 1: col2_1, 2: col2_2, 3:col2_3}

In [ ]:
for index, row in Data.iterrows():
    print('article_num',article_num)
    article_num+=1
    col1.append(row['mixed_doc'])
    topic_separated_summary = [[] for _ in np.arange(4)]
    Document= row['mixed_doc'].split("\n")

    health = ' '
    sports = ' '
    politics = ' '
    food = ' '
    if (type(row['health'])!=float and len(row['health'].split("\n"))>3):
      health = ' '.join(row['health'].split("\n"))
      # print('0',health)
    if (type(row['sports'])!=float and len(row['sports'].split("\n"))>3):
      sports =  " ".join(row['sports'].split("\n"))
      # print('1',sports)
    if (type(row['politics'])!=float and len(row['politics'].split("\n"))>3):
      politics = " ".join(row['politics'].split("\n"))
      # print('2',politics)
    if (type(row['food'])!=float and len(row['food'].split("\n"))>3):
      food = " ".join(row['food'].split("\n"))
      # print('3',food)
    # topic_separated_string = row['topic_separated_doc']
    # topic_separated_string = topic_separated_string.replace("[", "")
    # topic_separated_string = topic_separated_string.replace("]", "")
    topic_separated_doc = [health, sports, politics, food]

    # print(np.array(topic_separated_string).shape)
    # print(np.array(topic_separated_doc).shape)

    for i in np.arange(len(topic_separated_doc)):
      if topic_separated_doc[i]!=' ':
        # print(topic_separated_doc[i])
        article = str(" ".join(topic_separated_doc[i].split("\n")))
        # print('article',article)
        inputs = tokenizer.encode("summarize: " + article, return_tensors="pt", max_length=512, truncation=True)
        # generate the summarization output
        outputs = model.generate(
            inputs, 
            max_length=150, 
            min_length=40, 
            length_penalty=2.0, 
            num_beams=4, 
            early_stopping=True)
        # just for debugging
        # print(outputs)
        # print("Summary:" ,tokenizer.decode(outputs[0]))
        dic[i].append(tokenizer.decode(outputs[0]))
      else:
        dic[i].append(' ')
 
      
    # break

In [ ]:
print(np.array(col2_0).shape)
print(np.array(col2_1).shape)
print(np.array(col2_2).shape)
print(np.array(col2_3).shape)

In [ ]:
import pandas as pd
#{'Health':0, 'Sports':1, 'Politics':2, 'Food':3}
Data = pd.DataFrame(list(zip(col1,col2_0,col2_1,col2_2,col2_3)), 
               columns =['mixed_doc', 'health', 'sports', 'politics', 'food']) 
Data = Data.sample(frac = 1)
Data.head(5)


,mixed_doc,health,sports,politics,food
25,"BACKGROUND.\nAn MRI costs $1,145 in America an...",introduction of mass spectrometry provides mas...,the 19-year-old scored his fourth goal of a eu...,"a knee replacement costs $25,398 in the u.s. a...",CONCLUSIONS. RESULTS. CONCLUSIONS. RESULTS. CO...
17,giftThis was a gift for someone that loves pop...,,"created by Mason Gordon, SlamBall was born out...","the court has changed a bit since, but the fun...",a glass of wine goes nicely with Spicy Thai ch...
8,The BestAbsolutely the best French Roast k-cup...,it is estimated that approximately ⁇ dig> pla...,angel reyna scored a superb goal for Pachuca a...,.................................................,i've tried various weights and water temps to ...
48,Great Mustard2Love this product.\nVery flavora...,,the 1994 Chargers lost 49-26 to the 49ers in s...,the 1994 Chargers quarterback died of a drug o...,my 6-month-old is a fanatic for these flavors!...
30,"BACKGROUND.\nSafe, easy to chew, tasty (for do...",enterobacter sakazakii was defined as a new sp...,the first-time all-star was hurt in a fight th...,.......................,geriatric mini poodle can't chew biscuits or r...


In [ ]:
Data.to_csv("/content/drive/MyDrive/Anushree/done_summary_bert.csv")